<a href="https://colab.research.google.com/github/GuyNYAMSI/exemple_repos/blob/main/Compl%C3%A9ments_couches_denses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/logo_datascientest.png" style="height:150px;center"></center>

<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>Deep-Learning - Modules complémentaires</h1></center>
<center><h2>Compléments sur le dataset MNIST</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">


Le but de cet exercice est d'approfondir nos connaissances sur l'entraînement d'un modèle en deep learning.

 Nous allons travailler avec la base de données **```MNIST```**. Cette base contient des images de chiffres manuscrits que nous tenterons de classifier.

> Nous aborderons les points suivants : 
>> I - [Préparation du dataset](#preparation)
>>
>>
>> II - [Influence de la taille des batchs](#batch_size)
>>> A - [Pour nombre d'epoch fixé](#fixed_epochs)
>>>
>>>
>>> B - [Pour un nombre d'itérations fixé](#fixed_iterations)
>>
>>
>> III - [Illustration du surentraînement](#overfitting)
>>
>>
>> IV - [Influence du nombre d'images dans le jeu d'entraînement](#train_size)
>>
>>
>> V - [Influence de l'architecture du réseau de neurones](#architecture)
>>> A - [Architecture très basique VS architecture plus complexes](#dense)
>>>
>>>
>>> B - [Introduction aux réseaux de neurones convolutionnels](#CNN)

- (a) Exécuter la cellule ci-dessous pour importer les modules nécessaires à l'exercice ainsi que le dataset ```MNIST```.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

from keras.datasets.mnist import load_data
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

import time

(X_train, y_train), (X_test, y_test) = load_data()

- (b) Afficher, dans une grille de figures, 6 images tirées aléatoirement de l'échantillon ```X_train```, avec en titre les labels correspondants de ```y_train```.
> - Cet affichage a déjà été fait dans le premier notebook du cours.

# <a name="preparation"></a> I - Préparation du dataset

- (a) Changer la forme des images de ```X_train``` et de ```X_test``` en des vecteurs de tailles **28*28** à l'aide de la méthode ```reshape```.

- (b) Pour une meilleure performance du modèle, réduire les pixels des données ```X_train``` et ```X_test``` afin qu'ils soient compris entre 0 et 1. 
> - Diviser **l'ensemble de l'échantillon** par 255.

- (c) Transformer les labels de ```y_train``` et ```y_test``` en vecteurs catégoriels binaires (one hot) grâce à la fonction to_categorical du sous-module ```np_utils``` de ```keras```.

- (d) Extraire dans des variables appelées respectivement ```num_pixels``` et ```num_classes``` le nombre de colonnes (pixels) de ```X_train``` et le nombre de colonnes (classes) de ```y_test```.

# <a name="batch_size"></a> II - Influence de la taille des batchs



## <a name="fixed_epochs"></a> A - Nombre fixe d'epoch

- (a) Définir une liste ```batch_sizes``` contenant les valeurs ```[1, 20, 50, 200, 500, 2000, 4000]```.

Nous allons utiliser le modèle suivant :


```
  model = Sequential()
  model.add(Dense(units = 20, input_dim = num_pixels, kernel_initializer ='normal', activation ='tanh'))             
  model.add(Dense(units = num_classes, kernel_initializer ='normal', activation ='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
```



- (b) Pour chaque taille de batch défini dans ```batch_sizes```, définir et entraîner le modèle sur les données ```X_train``` et ```y_train``` grâce à la méthode ```fit``` :
>  - L'entraînement devra se faire sur 5 epochs (paramètre ```epochs```).
>
>  - La performance du modèle devra être évaluée sur un échantillon de validation contenant 20% des données (paramètre ```validation_split```).
>
>  - La sortie des entraînements devra être stockée dans une liste nommée ```training_histories_fixed_epoch```.

- (c) Stocker dans une liste ```times_fixed_epoch``` la durée de chaque entraînement.

- (d) Tracer sur un même graphe l'évolution de la précision d'entraînement en fonction de l'epoch pour chaque taille de batch.

- (e) Sur un second graphe, tracer le temps d'entraînement du modèle pour chaque taille de batch.

## <a name="fixed_iterations"></a> B - Nombre fixe d'itérations

- (f) Définir une liste ```batch_sizes_2``` contenant les valeurs ```[20, 50, 200, 500, 2000, 4000]```.

- (g) Définir une liste ```epochs``` contenant les valeurs de ```batch_size_2``` divisées par 5.
>  - Cela permettra d'entrainer le modèle sur un nombre fixe d'itération plutôt qu'un nombre fixe d'epochs.
>
>  - Attention à ce que les valeurs de ```epochs``` soient bien des valeurs entières.

- (h) Pour chaque taille de batch défini dans ```batch_sizes```, définir et entraîner le modèle sur les données ```X_train``` et ```y_train``` grâce à la méthode ```fit``` :
>  - L'entraînement devra se faire avec les éléments de la liste ```epoch``` (paramètre ```epochs```).
>
>  - La perfomance du modèle devra être évaluée sur un échantillon de validation contenant 20% des données (paramètre ```validation_split```).
>
>  - La sortie des entraînements devra être stockée dans une liste nommée ```training_histories_fixed_iterations```.

- (i) Stocker dans une liste ```times_fixed_iterations``` la durée de chaque entraînement.

- (j) Tracer sur un même graphe l'évolution de la précision d'entraînement en fonction des itérations pour chaque taille de batch.

- (k) Sur un second graphe, tracer le temps d'entraînement du modèle pour chaque taille de batch.

On remarque que plus on entraîne notre modèle, plus la précision d'entraînement augmente. On peut alors penser que plus on entraîne notre modèle, meilleure sera la prédiction.

Ceci est très loin de la vérité. C'est même un des problèmes les plus classiques en deep learning, qu'on appelle **le surentraînement** (ou **overfitting**)

# <a name="overfitting"></a> III - Illustration du surentraînement

- (a) Instancier le même modèle que lors de la partie précédente.

- (b) Entraîner le modèle sur les données ```X_train``` et ```y_train``` grâce à la méthode ```fit``` :
>  - L'entraînement devra se faire sur **100** epochs (paramètre ```epochs```).
>
>  - Les batchs devront avoir une taille de 20 (paramètre ```batch_size```)
>
>  - La perfomance du modèle devra être évaluée sur un échantillon de validation contenant 20% des données (paramètre ```validation_split```).
>
>  - La sortie de l'entraînement devra être stockée dans une variable nommée ```training_history```

- (c) Tracer sur un même graphe l'évolution des précisions d'entraînement et de validation. Que remarquez-vous?

# <a name="train_size"></a> IV - Influence du nombre d'images dans le jeu d'entraînement

- (a) Définir une liste ```train_sizes``` contenant les valeurs ```[10, 200, 1000, 5000, 15000, 60000]```.

- (b) Pour chaque élément n de ```train_sizes```, créer un échantillon de taille n de ```X_train``` et de ```y_train```. Stocker ces échantillons dans des listes ```X_train_samples``` et ```y_train_samples```.
>  - Attention à ce que les éléments sélectionnés soient les mêmes pour les échantillons de ```X_train``` et ```y_train```.

- (c) Pour chaque jeu d'entraînement, définir et entraîner le modèle que dans les parties précédentes grâce à la méthode ```fit``` :
>  - L'entraînement devra se faire sur 10 epochs (paramètre ```epochs```).
>  
>  - Les batchs devront avoir une taille de 20 (paramètre ```batch_size```)
>
>  - La perfomance du modèle devra être évaluée sur un échantillon de validation contenant 20% des données (paramètre ```validation_split```).
>
>  - La sortie des entraînements devra être stockée dans une liste nommée ```training_histories```.
  
- (d) Stocker dans une liste ```times``` la durée de chaque entraînement.

- (e) Tracer sur un même graphe l'évolution de la précision d'entraînement en fonction des epochs pour chaque jeu d'entraînement.

- (f) Sur un deuxième graphe, tracer l'évolution de la précision de validation en fonction des epochs pour chaque jeu d'entraînement.

- (g) Sur un dernier graphe, tracer le temps d'entraînement du modèle pour chaque jeu d'entraînement.

# <a name="architecture"></a> V - Influence de l'architecture du réseau de neurones

## <a name="dense"></a> A - Architecture très basique VS architecture plus complexe

Nous allons maintenant évaluer l'influence de l'architecture d'un modèle sur ses performances.
- (a) Exécuter la cellule suivante pour instancier le modèle très simple que nous avons utilisé dans les parties précédentes.

In [ ]:
model1 = Sequential()
model1.add(Dense(units = 20, input_dim = num_pixels, kernel_initializer ='normal', activation ='tanh'))             
model1.add(Dense(units = num_classes, kernel_initializer ='normal', activation ='softmax'))

model1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

- (b) Instancier un modèle appelé ```model2``` avec au moins trois couches denses (en comptant la couche finale). Vous êtes libres de choisir les paramètres des couches.

- (c) Entraîner les modèles grâce à la méthode ```fit``` :
>  - L'entraînement devra se faire sur 10 epochs (paramètre ```epochs```).
>  
>  - Les batchs devront avoir une taille de 20 (paramètre ```batch_size```)
>
>  - La perfomance du modèle devra être évaluée sur un échantillon de validation contenant 20% des données (paramètre ```validation_split```).
>
>  - La sortie des entraînements devra être stockée dans des variables appelées ```training_history_1``` et ```training_history_2```.

- (d) Tracer sur un même graphe l'évolution de la précision d'entraînement en fonction des epochs pour les deux modèles.

- (e) Sur un deuxième graphe, tracer l'évolution de la précision de validation en fonction des epochs pour les deux modèles.

Que constatez-vous?

## <a name="CNN"></a> B - Introduction aux réseaux de neurones convolutionnels


Les couches denses ne sont pas les seules couches que l'on peut créer sur python. Par exemple, on peut définir le réseau de convolution suivant (architecture **LeNet**), qui est bien plus adapté au traitement des images que les modèles définis précédemment.
- (f) Exécuter les cellules suivantes pour constater l'efficacité de l'architecture **LeNet**.

In [ ]:
X_train_conv = X_train.reshape((-1, 28, 28, 1))
X_test_conv = X_test.reshape((-1, 28, 28, 1))

model_conv = Sequential()
model_conv.add(Conv2D(filters=30, kernel_size=(5,5), padding='valid', input_shape=(28,28,1), activation='relu'))
model_conv.add(MaxPooling2D(pool_size=(2,2)))
model_conv.add(Conv2D(filters=16, kernel_size=(3,3), padding='valid', activation='relu'))
model_conv.add(MaxPooling2D(pool_size=(2,2)))
model_conv.add(Dropout(rate=0.2))
model_conv.add(Flatten())
model_conv.add(Dense(units = 128, activation ='relu'))             
model_conv.add(Dense(units = num_classes, activation ='softmax'))

model_conv.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
training_history_conv = model_conv.fit(X_train_conv, y_train, epochs = 10, batch_size = 20, validation_split = 0.2)

In [ ]:
plt.figure(figsize=(24,8))

plt.subplot(121)
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(np.arange(1 , 11, 1), training_history_1.history['accuracy'], label = 'model1', color='blue')
plt.plot(np.arange(1 , 11, 1), training_history_2.history['accuracy'], label = 'model2', color='red')
plt.plot(np.arange(1 , 11, 1), training_history_conv.history['accuracy'], label = 'model_conv', color='green')
plt.legend()

plt.subplot(122)
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(np.arange(1 , 11, 1), training_history_1.history['val_accuracy'], label = 'model1', color='blue')
plt.plot(np.arange(1 , 11, 1), training_history_2.history['val_accuracy'], label = 'model2', color='red')
plt.plot(np.arange(1 , 11, 1), training_history_conv.history['val_accuracy'], label = 'model_conv', color='green')
plt.legend();